In [1]:
!pip install PyPDF2 sentence-transformers

In [2]:
import PyPDF2
import nltk
from sentence_transformers import SentenceTransformer, util
import re
from tkinter import Tk, Button, Entry, Label, filedialog, Text, Scrollbar, Frame
from tkinter.ttk import Progressbar
import torch
import pandas as pd
from tqdm import tqdm
import os

device = torch.device("mps" if torch.has_mps else "cpu")
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

/Users/ahmadabdullah/anaconda3/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

/Users/ahmadabdullah/anaconda3/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/Users/ahmadabdullah/anaconda3/lib/python3.11/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feed

In [3]:
def read_pdf_files(pdf_paths):
    pdf_text_data = {}
    for pdf_path in pdf_paths:
        lines = []
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                page_text = page.extract_text()
                if page_text:
                    cleaned_text = re.sub(r'[^a-zA-Z0-9.,\s]\n', '', page_text)
                    lines += [line.strip() for line in re.split(r'[.,]', cleaned_text) if line.strip()]
        pdf_text_data[pdf_path] = lines
    return pdf_text_data

def find_citation(query_line, pdf_text_data, top_k=2):
    sub_queries = [sub.strip() for sub in query_line.split('. ') if sub.strip()]
    all_top_matches = []

    for sub_query in sub_queries:
        query_embedding = model.encode(sub_query, convert_to_tensor=True).to(device)
        sub_query_matches = []

        for pdf_path, lines in tqdm(pdf_text_data.items()):
            line_embeddings = model.encode(lines, convert_to_tensor=True).to(device)
            similarity_scores = util.cos_sim(query_embedding, line_embeddings)[0]
            
            top_indices = similarity_scores.topk(k=top_k).indices.tolist()
            
            for idx in top_indices:
                score = similarity_scores[idx].item()
                matched_line = lines[idx]
                sub_query_matches.append((score, pdf_path, matched_line, sub_query))
        
        sub_query_matches = sorted(sub_query_matches, key=lambda x: x[0], reverse=True)[:top_k]
        all_top_matches.extend(sub_query_matches)
    
    all_top_matches_df = pd.DataFrame(all_top_matches, columns=['Similarity', 'Path', 'Line', 'Sub_query'])
    
    return all_top_matches_df

In [4]:
file_paths = []
for file_name in os.listdir('../papers/'):
    if file_name.endswith('.pdf'):
        file_paths.append('../papers/' + file_name)

file_data = read_pdf_files(file_paths)

In [5]:
## Ibtehaj Literature Review

query = '''The research also showed that retracted papers were mainly written by 2-4
authors (38.7%), with single authors present in only 5% of retracted papers. It
also showed that 840 of the papers were original research papers, followed by
150 review papers.'''
find_citation(query ,file_data, top_k=1)

100%|█████████████████████████████████████████████| 9/9 [00:12<00:00,  1.34s/it]


,Similarity,Path,Line,Sub_query
0,0.874032,../papers/[5] ibto.pdf,Retracted papers were mainly written by ...,The research also showed that retracted papers...
1,0.699134,../papers/[5] ibto.pdf,000 papers published,It\nalso showed that 840 of the papers were or...


In [6]:
## Hocane Literature Review

query = 'They attempt to untan- gle various faces of plagiarism—presenting direct copying, self-plagiarism, and subtle forms like mosaic plagiarism—because each brings its own set of ethical concerns and consequences.'
find_citation(query ,file_data, top_k=1)

100%|█████████████████████████████████████████████| 9/9 [00:12<00:00,  1.37s/it]


,Similarity,Path,Line,Sub_query
0,0.694224,../papers/[4] hocane.pdf,which may be an indicator of self-plagiarism,They attempt to untan- gle various faces of pl...


In [7]:
## Hocane + Tahmooras Combined Literature Review

query = 'Ex- ploring 1082 papers from 2013 to 2016, the authors found that misconduct, such as plagiarism, false data manipulation, and fake peer review processes, resulted in 79% of papers being retracted, with plagiarism as the leading cause (32.7%), followed closely by data manipulation (32.5%)'
find_citation(query ,file_data, top_k=2)

100%|█████████████████████████████████████████████| 9/9 [00:12<00:00,  1.39s/it]


,Similarity,Path,Line,Sub_query
0,0.768673,../papers/[4] hocane.pdf,8% (89/213) were retracted for plagiarism,"Ex- ploring 1082 papers from 2013 to 2016, the..."
1,0.765789,../papers/[7] tams.pdf,Retractions in the scientiﬁc literatureis the ...,"Ex- ploring 1082 papers from 2013 to 2016, the..."


In [8]:
## Hocane + Tahmooras Combined Literature Review

query = '''This pressure is often described as the ”publish or perish” culture, where
career advancement is closely tied to publication records, to the point that some
researchers may compromise quality for quantity'''
find_citation(query ,file_data, top_k=2)

100%|█████████████████████████████████████████████| 9/9 [00:12<00:00,  1.38s/it]


,Similarity,Path,Line,Sub_query
0,0.701211,../papers/[9] ahmed.pdf,variations in the amount of external and inter...,This pressure is often described as the ”publi...
1,0.620849,../papers/[9] ahmed.pdf,Pressure to publish may also affect research i...,This pressure is often described as the ”publi...


In [10]:
## Burhan Combined Literature Review

query = '''Using the Medline database, they analyzed over 17 million published scientific studies, calculating the percentage of retractions by dividing the number of
retracted articles by the total number of published papers each year'''
find_citation(query ,file_data, top_k=3)

100%|█████████████████████████████████████████████| 9/9 [00:12<00:00,  1.37s/it]


,Similarity,Path,Line,Sub_query
0,0.796851,../papers/[1] burhan1.pdf,with scientific knowledge Fig 1 | Number of a...,"Using the Medline database, they analyzed over..."
1,0.789266,../papers/[1] burhan1.pdf,We used the Medline database to calcu late bot...,"Using the Medline database, they analyzed over..."
2,0.786642,../papers/[4] hocane.pdf,A major strength of this study was that retrac...,"Using the Medline database, they analyzed over..."


In [11]:
## Sharjeel Combined Literature Review

query = '''The study found that most retractions occurred in the
USA, while in terms of fraud retractions, India and Italy topped the list [6].
Using the Activity Index (AI), the author determined that the MULT (multidisciplinary sciences) field ranked highest in retractions. Garfield’s Impact
Factor (IF) was employed to gauge journal influence'''

find_citation(query ,file_data, top_k=2)

100%|█████████████████████████████████████████████| 9/9 [00:12<00:00,  1.35s/it]


,Similarity,Path,Line,Sub_query
0,0.675572,../papers/[6] sharj.pdf,The retraction\npatterns varied across differe...,The study found that most retractions occurred...
1,0.675367,../papers/[6] sharj.pdf,the retraction activityindex for USA was 1,The study found that most retractions occurred...
2,0.631180,../papers/[2] burhan2.pdf,348 journals with a known impact factor (IF),Garfield’s Impact\nFactor (IF) was employed to...
3,0.595812,../papers/[2] burhan2.pdf,(A) Higher- and lower-impact factor (IF) journ...,Garfield’s Impact\nFactor (IF) was employed to...


In [12]:
## Significance of Research

query = '''The legitimacy of any academic system is based on the integrity of its research,
which influences everything from public policy to technological innovation. The
growing tendency of research retractions and decline in Pakistan endangers the
foundation of the academic endeavor, potentially impacting researchers, students, and institutions.'''

find_citation(query ,file_data, top_k=2)

100%|█████████████████████████████████████████████| 9/9 [00:12<00:00,  1.36s/it]


,Similarity,Path,Line,Sub_query
0,0.599612,../papers/[9] ahmed.pdf,their impact on public trust in science and re...,The legitimacy of any academic system is based...
1,0.590128,../papers/[4] hocane.pdf,Scientific integrity,The legitimacy of any academic system is based...
2,0.631804,../papers/[9] ahmed.pdf,Academic retractions continue to be an issue,The\ngrowing tendency of research retractions ...
3,0.608910,../papers/[7] tams.pdf,Analysis of retractions puts spotlight on acad...,The\ngrowing tendency of research retractions ...
